# 準備

In [25]:
from dotenv import load_dotenv
load_dotenv() # .envファイルから環境変数を読み込み(OPENAI_API_KEY変数にAPI Keyが入っている)

True

# LLM Chain

In [26]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI() # これだけでモデルを初期化できるぽい

In [58]:
# これでモデルと会話することができる
print(llm.invoke("langsmithはどのようにテストを支援してくれますか？"))

content='Langsmithは、テストを支援するためにさまざまな機能を提供しています。以下にその一部を示します。\n\n1. テストケースの生成: Langsmithは、自動的にテストケースを生成する機能を提供します。開発者は、入力の範囲や条件を指定するだけで、複数のテストケースを生成することができます。\n\n2. テストスイートの作成: Langsmithは、テストスイートを作成するためのフレームワークを提供します。開発者は、テストケースをグループ化し、一括で実行することができます。\n\n3. テストの実行と結果の分析: Langsmithは、テストの実行と結果の分析を自動化します。開発者は、テストスイートを実行し、各テストケースの結果を確認することができます。また、テストカバレッジやパフォーマンスの情報も提供します。\n\n4. テストの自動化: Langsmithは、テストの自動化をサポートします。開発者は、テストスクリプトを作成し、定期的に実行することができます。これにより、継続的なテストが可能になります。\n\n5. テストの管理と報告: Langsmithは、テストの管理と報告を支援します。開発者は、テストケースやテストスイートを管理し、テスト結果やカバレッジのレポートを生成することができます。\n\nこれらの機能により、Langsmithは開発者にテストの効率化と品質向上を支援します。'


In [28]:
# プロンプトテンプレートを使うこともできる
from langchain_core.prompts import ChatPromptTemplate

# この例だとシステムに「技術ドキュメントライター」というロールを与えている
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは世界レベルの技術ドキュメントライターです。"),
    ("user", "{input}")
])

In [29]:
# 組み合わせてchainをつくる
chain = prompt | llm

In [30]:
# chainもinvokeで会話できる
# chain.invoke({"input": "langsmithはどのようにテストを支援してくれますか？"})

In [31]:
# 出力をAIMessageからStringに変える
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [32]:
# chainに加える
chain = prompt | llm | output_parser

In [33]:
# chain.invoke({"input": "langsmithはどのようにテストを支援してくれますか？"})

## Diving Deeper

さらに深く理解するためにたくさんのドキュメントが用意されている。今すぐにはとても読めないので、少しずつ読む。

# Retrieval Chain

In [34]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview") # URLを渡すとその内容を取ってきてくれる

docs = loader.load() # 指定したURLからドキュメントを取得する

In [35]:
from langchain_openai import OpenAIEmbeddings # ベクターストアに格納するためにドキュメントをベクトル化するためのモジュール

embeddings = OpenAIEmbeddings()

# これを使って、今からベクターストアにドキュメントを挿入する

In [36]:
# 簡単のため、シンプルなローカルベクターストアであるFAISSを使う
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter() # チャンクサイズの制限を下回るまで再帰的にテキストを分割するTextSplitter
documents = text_splitter.split_documents(docs) # TextSpritterに書けてテキストを分割する
vector = FAISS.from_documents(documents, embeddings) # 分割した文書をベクトル化してベクターストアに保存する

In [37]:
documents

[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, 

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain 

# 与えたコンテキストに基づいて質問に回答させるためのプロンプト
prompt = ChatPromptTemplate.from_template("""与えられた文脈のみに基づいて質問に回答してください:
<context>
{context}
</context>

質問: {input}""")

# 質問と取得したドキュメントを受け取り、解答を生成するチェーンを作成する
# create_stuff_documents_chainはドキュメントのリストをコンテキストとしてプロンプトに渡す
document_chain = create_stuff_documents_chain(llm, prompt)

In [39]:
from langchain_core.documents import Document 

# ドキュメントを自分で渡すことで、チェーンを自分で実行することもできる
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

In [40]:
# でも、retrieverを使うことで、質問に最も関連性の高いドキュメントを動的に選択してモデルに渡すことができる
from langchain.chains import create_retrieval_chain # retrieval chainを作るための関数

retriever = vector.as_retriever() # retrieverを作成
retrieval_chain = create_retrieval_chain(retriever, document_chain) # retrieverとドキュメントを受け取るchainからretrieval chainを作成

In [41]:
# これで、retrieval chainを呼び出すことができる
response = retrieval_chain.invoke({"input": "LangSmithはどのようにテストを支援してくれますか?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmithは、テストを支援するためのいくつかの機能を提供しています。まず、LangSmithはデータセットのアップロードを簡素化し、プロンプトやチェーンの変更をテストするために使用できます。データポイントのチェーン実行結果を視覚化し、出力を目視で確認することができます。また、テストランが完了した後に結果を評価するための評価者も提供されています。

さらに、LangSmithは人間の評価を行うための機能も提供しています。アノテーションキューを使用して、モデルタイプや自動評価スコアなどの基準に基づいてテストランを選択し、人間のレビューのためにキューに追加することができます。レビューアは、入力、出力、既存のタグを表示してからフィードバックを追加することができます。これにより、主観的な品質や自動評価指標では評価しづらい要素を評価したり、自動評価メトリックが信頼性の高い情報を捉えているかを確認するためのサンプリングと検証を行うことができます。

LangSmithは、テストを実施し、品質と信頼性を向上させるための人間のレビューやフィードバックの収集を容易にします。


## Diving Deeper

これも少しずつ読む

# Conversation Retrieval Chain

ここまでやってきたのは1つの質問に答えるchain。でもこれだと一問一答で会話ができない。フォローアップクエスチョンにもこたえられるchainを作るにはどうすればいいだろうか。

In [42]:
# create_retrieval_chain()をまだ使うことができる。でも変更を加える必要がある
# 1. 直近の入力だけでなく、すべての会話の歴史を入力に加える
# 2. 最後のLLM chainも同様にすべての会話の歴史を受け取って出力を生成する

from langchain.chains import create_history_aware_retriever # 歴史を考慮したretriever
from langchain_core.prompts import MessagesPlaceholder # メッセージプレースホルダー (何なのかよくわからないぞ) 

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"), # メッセージプレースホルダの中にchat_historyを入れる
    ("user", "{input}"), # ユーザのインプット
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
    # 以上を踏まえて、会話に関係する情報を取得するために検索する検索クエリを生成してください
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt) # 歴史を踏まえた生成を行うchainを生成

In [43]:
from langchain_core.messages import HumanMessage, AIMessage # 人間のメッセージとAIのメッセージ

# 仮の歴史
chat_history = [HumanMessage(content="LangSmithは私のLLMアプリケーションのテストを支援してくれますか？"), AIMessage(content="もちろんです!")]

# chainに入力して続きを出力
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "どのように支援してくれるのですか"
})

# LLMはクエリを生成するため、LangSmithを使ったテストに関するドキュメントを返す(なんでクエリを生成したらドキュメントが返ってくるのかわかんない)

[Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by setting environment variables, which we establish whenever we launch a virtual environment or open our bash shell and leave them set. The same principle app

In [44]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "以下のコンテキストを参照してユーザーの質問に答えてください:\n\n{context}"), # コンテキスト
    MessagesPlaceholder(variable_name="chat_history"), # 会話の歴史
    ("user", "{input}"), # ユーザーの入力
])
document_chain = create_stuff_documents_chain(llm, prompt) # 変更したプロンプトで再びchainを生成

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain) # 歴史も参照できるchainを生成

In [45]:
chat_history = [HumanMessage(content="LangSmithは私のLLMアプリケーションのテストを支援してくれますか?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "どのように支援してくれますか？"
})

{'chat_history': [HumanMessage(content='LangSmithは私のLLMアプリケーションのテストを支援してくれますか?'),
  AIMessage(content='Yes!')],
 'input': 'どのように支援してくれますか？',
 'context': [Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
  Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work nee

# Agent

LLMがどのような手順を実行するのかを決定する

In [56]:
# まずはエージェントがどのツールにアクセスできるのかを定義する
from langchain.tools.retriever import create_retriever_tool # 今までのはcreate_retrieval_chainだった

retriever_tool = create_retriever_tool(
    retriever, # 会話の履歴を取ってくるもの
    "langsmith_search", # ツールの名前?
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!", # このツールの使い方をモデルに説明してる
    # ↑ LangSmithに関する情報を探せ。LangSmithに関する質問が来たら、このツールを使わなくてはなりませんよ！
)

In [57]:
# 検索ツールとしてTavilyを使う
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults() # エージェントが使う検索ツール

In [50]:
# エージェントが使うツールのリスト
tools = [retriever_tool, search]

In [51]:
from langchain_openai import ChatOpenAI # LLMモデルをロードするためのモジュール
from langchain import hub # プロンプトをpullするためのモジュール
from langchain.agents import create_openai_functions_agent # エージェントを作るためのモジュール
from langchain.agents import AgentExecutor # エージェントを作るためのモジュール

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent") # プロンプトをpullしてくる
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) # モデルを指定してAPIをロード
agent = create_openai_functions_agent(llm, tools, prompt) # モデル、ツール、プロンプトをまとめて渡し、エージェントを初期化
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # エージェントを実行する準備を完了

In [52]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.

4. Test Prioritization: LangSmith can prioritize the exec

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.\n\n4. Test Prioritization: Lan

In [53]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://weatherspark.com/h/m/557/2024/1/Historical-Weather-in-January-2024-in-San-Francisco-California-United-States', 'content': 'January 2024 Weather History in San Francisco California, United States  Daily Precipitation in January 2024 in San Francisco Observed Weather in January 2024 in San Francisco  San Francisco Temperature History January 2024 Hourly Temperature in January 2024 in San Francisco  Hours of Daylight and Twilight in January 2024 in San Francisco13.8 mph moderate breeze Wind Dir. 70 deg, ENE Cloud Cover Mostly Cloudy 15,000 ft Mostly Clear 3,000 ft Mostly Cloudy 18,000 ft Raw: KSFO 150756Z 07012KT 10SM FEW030 BKN150 BKN180 11/08 A3015 RMK AO2 SLP208 T01060083 401500100 This report shows the past weather for San Francisco, providing a weather history for January 2024.'}, {'url': 'https://www.whereandwhen.net/when/north-america/cali

{'input': 'what is the weather in SF?',
 'output': 'I found some information about the weather in San Francisco. In January 2024, the weather in San Francisco was mostly cloudy with a moderate breeze. The average temperature ranged from 41°F to 54°F. If you would like more detailed information, you can visit the following websites:\n\n1. [WeatherSpark](https://weatherspark.com/h/m/557/2024/1/Historical-Weather-in-January-2024-in-San-Francisco-California-United-States)\n2. [WhereAndWhen](https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/january/)\n3. [Climate-Data.org](https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/january-1/)\n\nPlease note that these websites provide historical weather data for January 2024 in San Francisco.'}